In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset

/kaggle/input/train-and-test/train.csv
/kaggle/input/train-and-test/test.csv


In [3]:
df = pd.read_csv("/kaggle/input/train-and-test/train.csv")


columns_to_drop = ["Cabin", "Name", "Embarked", "Fare", "Ticket", "PassengerId", "Age"]
df = df.drop(columns=columns_to_drop)
df["Sex"] = df["Sex"].map({"male": 1, "female": 0}) 
y_train = df["Survived"]
X_train = df.drop(columns=["Survived"])


In [4]:
X_test1 = pd.read_csv("/kaggle/input/train-and-test/test.csv")
X_test = X_test1.drop(columns=columns_to_drop)
X_test["Sex"] = X_test["Sex"].map({"male": 1, "female": 0})
X_test


,Pclass,Sex,SibSp,Parch
0,3,1,0,0
1,3,0,1,0
2,2,1,0,0
3,3,1,0,0
4,3,0,1,1
...,...,...,...,...
413,3,1,0,0
414,1,0,0,0
415,3,1,0,0
416,3,1,0,0


In [5]:
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor)
print(X_train.shape)  # Должно быть (n_samples, 4)
print(X_test.shape) 

(891, 4)
(418, 4)


In [6]:
from torch import nn
model = nn.Sequential(
    nn.Linear(in_features=4, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=64),
    nn.ReLU(),
    nn.Linear(in_features=64, out_features=1),
    nn.Sigmoid(),
    nn.Flatten(start_dim=0)
)
loss_function = nn.BCELoss()

from torch import optim

# your code here
optimizer = optim.SGD(model.parameters(), lr=1e-1)



def run(model, dataloader, loss_function, optimizer=None):
    # set the model to evaluation or training mode
    if optimizer == None:
        model.eval()
    else:
        model.train()

    total_loss = 0

    for X, y in dataloader:
        # compute prediction
        pred = model(X)
        # compute loss
        loss = loss_function(pred, y)
        # save loss
        total_loss += loss.item()
        if optimizer != None:
            # compute gradients
            loss.backward()
            # do optimizer step
            optimizer.step()
            # clear gradients
            optimizer.zero_grad()

    return model.parameters()
NUM_EPOCHS = 20
BATCH_SIZE = 20
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)
for i in range(NUM_EPOCHS):
    run(model, train_loader, loss_function, optimizer)
y_test_pred = []
model.eval()  
with torch.no_grad(): 
    for X_batch in test_loader:
    
        X_batch = X_batch[0]
        pred = model(X_batch) 
        y_test_pred.append(pred)
y_test_pred = torch.cat(y_test_pred, dim=0)
y_test_pred_rounded = torch.round(y_test_pred)
y_test_pred_int = y_test_pred_rounded.int()



In [7]:
output = pd.DataFrame({'PassengerId': X_test1.PassengerId, 'Survived': y_test_pred_int})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
